In [ ]:
#Imports --StackOverflow csv data dump not included--
import pandas as pd
import re
import dask
import dask.dataframe as dd
from sklearn import svm
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.collocations import *
from collections import defaultdict
nltk.download('punkt')
nltk.download('wordnet')
import numpy as np
from glob import glob

#Processing methods

#distinct number of instances
def intersection_without_dup(body, words,postid,feature_container): 
    body_set = set(body) 
    words_set = set(words) 
    inter_set = set()
    if (body_set & words_set): 
        #print(body_set & words_set)
        inter_set = body_set & words_set
        feature_container[postid] = len(inter_set)
    

    #total number of instances
def intersection_with_dup(body_list, words_list,postid,feature_container): 
    inter_list = [value for value in body_list if value in words_list] 
    if (len(inter_list)>0):
        feature_container[postid] = len(inter_list)
   


def preprocess(text):
    stop_words = set(stopwords.words('english')) 
    punctuation = [',', '.','@','``',"''",'#',"'",'&','...',':',';']
    word_tokens=word_tokenize(text)
    result = []
    for token in word_tokens:
        result.append(token.lower())
        
    filtered_result = [w for w in result if not w in stop_words]
    ffiltered_result = [w for w in filtered_result if not w in punctuation]
    fffiltered_result = [w for w in ffiltered_result if not len(w)<3]
    return fffiltered_result

In [ ]:
#Query the Stackoverflow data dump for mentions of technical debt in post body, title or tags.


def process_pd(merged_pd):
    
    merged_pd=pd.DataFrame(np.vstack([merged_pd.columns, merged_pd]))
    merged_pd=merged_pd.rename(columns={0: 'PostId',1: 'PostTypeId', 3: 'ParentId',4: 'Creation',6:'Score', 7:'ViewCount', 8: 'Body', 15: 'Title', 16: 'Tags', 17:'AnswerCount', 18:'CommentCount', 19: 'FavoriteCount'})
    mask = (merged_pd['PostTypeId'] == 2.0) & (merged_pd['ParentId'] != 0)
    merged_pd['PostId'][mask] = merged_pd['ParentId']
    merged_pd['Body'] = merged_pd['Body'].str.replace('&lt;p&gt;','')
    merged_pd['Body'] = merged_pd['Body'].str.replace('&lt;','')
    merged_pd['Body'] = merged_pd['Body'].str.replace('&gt;','')
    merged_pd['Body'] = merged_pd['Body'].str.replace('&xA;','')
    merged_pd['Body'] = merged_pd['Body'].str.replace('/p&xA;','')
    merged_pd['Body'] = merged_pd['Body'].str.replace('/ul','')
    merged_pd['Body'] = merged_pd['Body'].str.replace('/li','')
    merged_pd['Body'] = merged_pd['Body'].str.replace('h1','')
    merged_pd['Body'] = merged_pd['Body'].str.replace('h2','')
    merged_pd['Body'] = merged_pd['Body'].str.replace('h3','')
    merged_pd['Body'] = merged_pd['Body'].str.replace('/strong','')
    merged_pd['Body'] = merged_pd['Body'].str.replace('&quot;','')
    merged_pd['Body'] = merged_pd['Body'].str.replace('/p&#xA;&#xA;','')
    merged_pd['Body'] = merged_pd['Body'].str.replace(' &#xA;','')
    merged_pd['Body'] = merged_pd['Body'].fillna('')
    merged_pd['Title'] = merged_pd['Title'].str.replace('&quot;','')
    merged_pd['ParentId'] = merged_pd['ParentId'].fillna('')
    merged_pd['Title'] = merged_pd['Title'].fillna('')
    merged_pd['Tags'] = merged_pd['Tags'].str.replace('&lt;p&gt;','')
    merged_pd['Tags'] = merged_pd['Tags'].str.replace('&lt;',' ')
    merged_pd['Tags'] = merged_pd['Tags'].str.replace('&gt;',' ')
    merged_pd['Tags'] = merged_pd['Tags'].str.replace('&xA;',' ')
    merged_pd['Tags'] = merged_pd['Tags'].str.replace('/p&xA;','')
    merged_pd['Tags'] = merged_pd['Tags'].fillna('')
    merged_pd['Score'] = merged_pd['Score'].fillna(0)
    merged_pd['ViewCount'] = merged_pd['ViewCount'].fillna(0)
    merged_pd['AnswerCount'] = merged_pd['AnswerCount'].fillna(0)
    merged_pd['CommentCount'] = merged_pd['CommentCount'].fillna(0)
    merged_pd['FavoriteCount'] = merged_pd['FavoriteCount'].fillna(0)
    merged_pd['Body'] = merged_pd['Body'].str.replace('href=','')
    merged_pd['Body'] = merged_pd['Body'].str.replace('strongem','')
    merged_pd['Body'] = merged_pd['Body'].str.replace('blockquote','')

    return merged_pd

def check_pd_body(merged_pd):
    
    return merged_pd[merged_pd['Body'].str.contains('technical debt|Technical debt|Technical Debt|technical debts|Technical debts|technical-debt|tech debt|tech-debt|techdebt')]

def check_pd_title(merged_pd):
    
    return merged_pd[merged_pd['Title'].str.contains('technical debt|Technical debt|Technical Debt|technical debts|Technical debts|technical-debt|tech debt|tech-debt|techdebt')]

def check_pd_tags(merged_pd):
    
    return merged_pd[merged_pd['Tags'].str.contains('technical-debt|tech-debt|techdebt')]


In [ ]:
#Read the parts after translating the stack overflow dump from xml to csv

names=[]


for name in sorted(glob('posts2_csv/*')):
    names.append(name)
    
def query_tech_debt(df):
    newDF = pd.DataFrame()
    newDF_title = check_pd_title(process_pd(df))
    newDF_body = check_pd_body(process_pd(df))
    newDF_tags = check_pd_tags(process_pd(df))
    pids=[]
    df = process_pd(df)

    
    if(newDF_title.empty == False) or (newDF_body.empty == False) or (newDF_tags.empty == False):
        for pid in newDF_title['PostId']:
            pids.append(pid)
        for pid in newDF_body['PostId']:
            pids.append(pid)
        for pid in newDF_tags['PostId']:
            pids.append(pid)
        for pid in pids:
            newDF=newDF.append(df.loc[df['PostId'] == pid, ['PostId','PostTypeId','Title','Body','Score','ViewCount','CommentCount',  'FavoriteCount']])
    
   
    return newDF.drop_duplicates(keep='first')
        
    
       
final_query_df=pd.DataFrame()
title_df=pd.DataFrame()



for n in names[2000:]:
    read_df=pd.read_csv(n)
    final_query_df=final_query_df.append(query_tech_debt(read_df))
    title_df=title_df.append(process_pd(read_df))



In [ ]:
mask=final_query_df['PostTypeId'] == 1  
final_query_df=final_query_df[mask]
title_df= title_df[['PostId','ParentId', 'Title']]


title_df=title_df.loc[title_df['ParentId'] == '', ['Title','PostId']]
title_df= title_df.iloc[1:]
title_df['PostId']=title_df['PostId'].fillna(0)
title_df['PostId']=title_df['PostId'].astype(int)
title_df=title_df.set_index('PostId')
title_df = title_df.loc[~title_df.index.duplicated(keep='first')]


final_query_df['PostId']=final_query_df['PostId'].fillna(0)
final_query_df['PostId']=final_query_df['PostId'].astype(int)
final_query_df=final_query_df.set_index('PostId')

#add titles to questions and answers
for ind in final_query_df.index:
            if(ind in final_query_df.index.values and ind in title_df.index.values):
                final_query_df.loc[[ind], 'Title']= title_df.loc[[ind], 'Title']
            

In [ ]:
# TDR CLASSIFIER FEATURES


# loading the tech debt labeled dataset
tech_labeled_data=pd.read_csv("tech_debt_labeled.csv")
tech_labeled_body_nocode = pd.read_csv("query_nocode.csv")
tech_labeled_data['Body'] = tech_labeled_body_nocode['Body']
tech_labeled_data['TDTYPE'] = tech_labeled_data['TDTYPE'].fillna('')
tech_labeled_data['URGENCY'] = tech_labeled_data['URGENCY'].fillna('0')



#make new dictionaries for features
#tokens
body_tech_labeled_bigrams= defaultdict()
title_tech_labeled_bigrams= defaultdict()
body_tech_labeled_words= defaultdict()
title_tech_labeled_words= defaultdict()

#frequencies for bodies: distinct, total and all words
body_tech_labeled_bigrams_total = defaultdict()
body_tech_labeled_words_total=defaultdict()
body_tech_labeled_bigrams_distinct = defaultdict()
body_tech_labeled_words_distinct=defaultdict()
body_total_tech_words = defaultdict()

#frequencies for titles: distinct, total and all words
title_tech_labeled_bigrams_total = defaultdict()
title_tech_labeled_bigrams_distinct = defaultdict()
title_tech_labeled_words_total=defaultdict()
title_tech_labeled_words_distinct=defaultdict()
title_total_tech_words = defaultdict()



#find bigrams and unigrams in body and title of posts and associate with postID
for ind in tech_labeled_data.index: 
    body_tech_labeled_bigrams[tech_labeled_data['PostId'][ind]] = list(nltk.bigrams(preprocess(str(tech_labeled_data['Body'][ind]))))
    body_tech_labeled_words[tech_labeled_data['PostId'][ind]] = preprocess(str(tech_labeled_data['Body'][ind]))
    
for ind in tech_labeled_data.index: 
    title_tech_labeled_bigrams[tech_labeled_data['PostId'][ind]] = list(nltk.bigrams(preprocess(str(tech_labeled_data['Title'][ind]))))
    title_tech_labeled_words[tech_labeled_data['PostId'][ind]] = preprocess(str(tech_labeled_data['Title'][ind]))
    
    
for postid,bgrm in body_tech_labeled_bigrams.items():
        intersection_with_dup(bgrm,debt_list,postid,body_tech_labeled_bigrams_total)
        intersection_without_dup(bgrm,debt_list,postid,body_tech_labeled_bigrams_distinct)
        
for postid,word in  body_tech_labeled_words.items():
        intersection_with_dup(word,word_freq,postid,body_tech_labeled_words_total)
        intersection_without_dup(word,word_freq,postid,body_tech_labeled_words_distinct)
        body_total_tech_words[postid] = word
        
for postid,bgrm in title_tech_labeled_bigrams.items():
        intersection_with_dup(bgrm,debt_list,postid,title_tech_labeled_bigrams_total)
        intersection_without_dup(bgrm,debt_list,postid,title_tech_labeled_bigrams_distinct)
        
for postid,word in  title_tech_labeled_words.items():
        intersection_with_dup(word,word_freq,postid,title_tech_labeled_words_total)
        intersection_without_dup(word,word_freq,postid,title_tech_labeled_words_distinct)
        
        
for postid,words in body_total_tech_words.items():
     body_total_tech_words[postid] = len(words)

In [ ]:
#TFIDF

tfidf_vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
tfidf_body = tfidf_vectorizer.fit_transform(tech_labeled_data['Body']).toarray()
tfidf_title= tfidf_vectorizer.fit_transform(tech_labeled_data['Title']).toarray()
tech_labeled_data['TFIDFBODY'] = [np.sum(arr) for arr in tfidf_body]
tech_labeled_data['TFIDFTITLE'] = [np.sum(arr) for arr in tfidf_title]


In [ ]:
#features added to csv dataframe

bbt=pd.DataFrame(body_tech_labeled_bigrams_total.items(), columns=['PostId', 'BodyTotalTechBigrams'])
bbd=pd.DataFrame(body_tech_labeled_bigrams_distinct.items(), columns=['PostId', 'BodyDistinctTechBigrams'])
btw = pd.DataFrame(body_tech_labeled_words_total.items(), columns=['PostId', 'BodyTotalTechWords'])
bdw = pd.DataFrame(body_tech_labeled_words_distinct.items(), columns=['PostId', 'BodyDistinctTechWords'])
bwt = pd.DataFrame(body_total_tech_words.items(), columns=['PostId', 'TotalBodyWords'])
tbt=pd.DataFrame(title_tech_labeled_bigrams_total.items(), columns=['PostId', 'TitleTotalTechBigrams'])
tbd=pd.DataFrame(title_tech_labeled_bigrams_distinct.items(), columns=['PostId', 'TitleDistinctTechBigrams'])

tech_labeled_data=pd.merge(tech_labeled_data, bbt, how='outer', on='PostId')
tech_labeled_data=pd.merge(tech_labeled_data, bbd, how='outer', on='PostId')
tech_labeled_data=pd.merge(tech_labeled_data, btw, how='outer', on='PostId')
tech_labeled_data=pd.merge(tech_labeled_data, bdw, how='outer', on='PostId')
tech_labeled_data=pd.merge(tech_labeled_data, bwt, how='outer', on='PostId')
tech_labeled_data=pd.merge(tech_labeled_data, tbt, how='outer', on='PostId')
tech_labeled_data=pd.merge(tech_labeled_data, tbd, how='outer', on='PostId')


In [ ]:

#Fill outer merge nans
tech_labeled_data['BodyTotalTechBigrams'] = tech_labeled_data['BodyTotalTechBigrams'].fillna('0')
tech_labeled_data['BodyDistinctTechBigrams'] = tech_labeled_data['BodyDistinctTechBigrams'].fillna('0')
tech_labeled_data['BodyTotalTechWords'] = tech_labeled_data['BodyTotalTechWords'].fillna('0')
tech_labeled_data['BodyDistinctTechWords'] = tech_labeled_data['BodyDistinctTechWords'].fillna('0')
tech_labeled_data['TotalBodyWords'] = tech_labeled_data['TotalBodyWords'].fillna('0')
tech_labeled_data['TitleTotalTechBigrams'] = tech_labeled_data['TitleTotalTechBigrams'].fillna('0')
tech_labeled_data['TitleDistinctTechBigrams'] = tech_labeled_data['TitleDistinctTechBigrams'].fillna('0')
tech_labeled_data['TDR'] = tech_labeled_data['TDR'].fillna('0')
tech_labeled_data['TDR'] = tech_labeled_data['TDR'].astype(int)
tech_labeled_data['TFIDFBODY']=tech_labeled_data['TFIDFBODY'].fillna(0)
tech_labeled_data['TFIDFTITLE']=tech_labeled_data['TFIDFTITLE'].fillna(0)
tech_labeled_data['TFIDFBODY']=tech_labeled_data['TFIDFBODY'].astype(int)
tech_labeled_data['TFIDFTITLE']=tech_labeled_data['TFIDFTITLE'].astype(int)

#Add scores and comments
scores_comments = pd.read_csv('scores_comments.csv')
scores_comments = scores_comments[['PostId','Score', 'CommentCount']]
tech_labeled_data=pd.merge(tech_labeled_data, scores_comments, how='outer', on='PostId')
tech_labeled_data['Score'] = tech_labeled_data['Score'].fillna(0)
tech_labeled_data['Score'] = tech_labeled_data['Score'].astype(int)
tech_labeled_data['CommentCount'] = tech_labeled_data['CommentCount'].fillna(0)
tech_labeled_data['CommentCount'] = tech_labeled_data['CommentCount'].astype(int)

In [ ]:
# TDR CLASSIFICATION
import joblib
from sklearn import datasets 
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn as sns

#Splitting the data into independent and dependent variables
X = tech_labeled_data[['TFIDFBODY','BodyDistinctTechBigrams','BodyDistinctTechWords','Score']]
y = tech_labeled_data['TDR']

X_train, X_test, y_train,y_test = train_test_split(X, y, test_size = 0.4, random_state = 0)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

clf = RandomForestClassifier(bootstrap=True,max_features=None, max_leaf_nodes=None,min_impurity_decrease=0.0,max_depth=24, random_state=42,criterion='entropy',min_samples_leaf=1, min_samples_split=2)

# Create the model on train dataset

model = clf.fit(X_train, y_train)
 
# Calculate the accuracy
print(accuracy_score(y_test, model.predict(X_test), normalize=True)*100)

# KFold Cross Validation approach
kf = KFold(n_splits=10,shuffle=False)
kf.split(X)    
     
# Initialize the accuracy of the models to blank list. The accuracy of each model will be appended to this list
accuracy_model = []
 
# Iterate over each train-test split
for train_index, test_index in kf.split(X):
    # Split train-test
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Train the model
    model = clf.fit(X_train, y_train)
    # Append to accuracy_model the accuracy of the model
    accuracy_model.append(accuracy_score(y_test, model.predict(X_test), normalize=True)*100)

# Print the accuracy    
print(accuracy_model)



# Visualize accuracy for each iteration
 
scores = pd.DataFrame(accuracy_model,columns=['Scores'])
 
sns.set(style="white", rc={"lines.linewidth": 3})
sns.barplot(x=['Iter1','Iter2','Iter3','Iter4','Iter5','Iter6','Iter7','Iter8','Iter9','Iter10'],y="Scores",data=scores)
plt.show()
sns.set()
print(confusion_matrix(y_test,model.predict(X_test)))
print(classification_report(y_test,model.predict(X_test)))

In [ ]:
#Technical Debt Urgency multiclass
from imblearn.over_sampling import SMOTE,RandomOverSampler

X = tdru[['TFIDFBODY','BodyDistinctTechBigrams','BodyDistinctTechWords']]
y=tdru['URGENCY']

over = SMOTE(random_state=12,sampling_strategy='all')
#over = RandomOverSampler(random_state=0,sampling_strategy='all')
X, y = over.fit_resample(X, y)

X_train, X_test, y_train,y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)



clf = RandomForestClassifier(bootstrap=True,max_features=None, max_leaf_nodes=None,min_impurity_decrease=0.0,max_depth=24, random_state=42,criterion='entropy',min_samples_leaf=1, min_samples_split=2)

# Create the model on train dataset

model = clf.fit(X_train, y_train)
 

from sklearn import metrics
print(metrics.classification_report(y_test, model.predict(X_test), target_names=['None','Very mild','Mild','Moderate','Severe','Very Severe']))



In [ ]:
#URGENCY TRINARY
#import SMOTE
from imblearn.over_sampling import SMOTE,RandomOverSampler

tdru['URGENCY'] = tdru['URGENCY'].map({1:0,2:1, 3:1, 4:2,5:2})
tdru['URGENCY']=tdru['URGENCY'].fillna(0)
tdru['URGENCY']=tdru['URGENCY'].astype(int)

#Score can also be added
X = tdru[['TFIDFBODY','BodyDistinctTechBigrams','BodyDistinctTechWords']]
y=tdru['URGENCY']

over = SMOTE(sampling_strategy='all')


X_train, X_test, y_train,y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

X_train, y_train= over.fit_resample(X_train, y_train)

clf = RandomForestClassifier(bootstrap=True,max_features=None, max_leaf_nodes=None,min_impurity_decrease=0.0,max_depth=70, random_state=0,criterion='entropy',min_samples_leaf=1, min_samples_split=2)

# Create the model on train dataset

model = clf.fit(X_train, y_train)
 
# Calculate the accuracy

print(metrics.classification_report(y_test, model.predict(X_test), target_names=['None-Very mild','Mild-Moderate','Severe-Very Severe']))


In [ ]:
#factorizing the td types and combining over/undersampling


#Score feature can also be added
XX = tdrs[['TFIDFBODY','BodyDistinctTechBigrams','BodyDistinctTechWords']]
yy = tdrs['TDTYPENUM']

over_smote = SMOTE(sampling_strategy='auto')

#TDTYPE RANDOM FOREST

XX_train, XX_test, yy_train,yy_test = train_test_split(XX, yy, test_size = 0.33, random_state = 0)

XX_train, yy_train= over_smote.fit_resample(XX_train, yy_train)


clf = RandomForestClassifier(bootstrap=True,max_features=None, max_leaf_nodes=None,min_impurity_decrease=0.0,max_depth=70, random_state=0,criterion='entropy',min_samples_leaf=1, min_samples_split=2)

# Create the model on train dataset

model = clf.fit(XX_train, yy_train)
 

print(metrics.classification_report(yy_test, model.predict(XX_test), target_names=[t for t in types]))



